In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import googlemaps
from networkx.algorithms.approximation import traveling_salesman_problem

gmaps = googlemaps.Client(key='API_KEY_SECRET')

locations = []

# Function to get elevation for a location using the Elevation API
def get_elevation(lat_lng):
    try:
        elevation_result = gmaps.elevation(lat_lng)
        if elevation_result:
            return elevation_result[0]['elevation'] # elevation in meters
    except Exception as e:
        print(f"Error fetching elevation for {lat_lng}: {e}")
    return None

# Dynamic minimum fuel consumption based on distance
def get_min_fuel_consumption(distance_km, fuel_efficiency_per_100km):
    if distance_km < 5:
        return max((distance_km * fuel_efficiency_per_100km) / 100, 0.1) # no less than 0.1 liters for very short distances
    return (distance_km * fuel_efficiency_per_100km) / 100

# Function to apply long-distance optimization
def apply_long_distance_optimization(distance_km, fuel_consumption, efficiency_threshold=50,reduction_factor=0.2):
    if distance_km > efficiency_threshold:
        print(f"Applying long-distance optimization for distance {distance_km} km (reduction factor: {reduction_factor*100}%)")
        return fuel_consumption * (1 - reduction_factor)
    return fuel_consumption

# Function to get travel time, distance, and fuel efficiency from GMAPS API
def get_real_time_data(origin_coords, destination_coords, metric, fuel_efficiency_per_100km=6.0):
    try:
        print(f"Fetching data for {origin_coords} to {destination_coords} (Metric: {metric})")
        result = gmaps.distance_matrix(origin_coords, destination_coords, mode='driving', departure_time='now')
        if result['rows'][0]['elements'][0]['status'] == 'OK':
            travel_time = result['rows'][0]['elements'][0]['duration_in_traffic']['value'] / 60
            travel_distance = result['rows'][0]['elements'][0]['distance']['value'] / 1000 # km

            if metric == 'time':
                return max(travel_time, 0)
            elif metric == 'distance':
                return max(travel_distance,0)
            elif metric == 'fuel':
                base_fuel_consumption = get_min_fuel_consumption(travel_distance, fuel_efficiency_per_100km)
                origin_elevation = get_elevation(origin_coords)
                destination_elevation = get_elevation(destination_coords)
                if origin_elevation is not None and destination_elevation is not None:
                    slope = (destination_elevation - origin_elevation) / travel_distance if travel_distance > 0 else 0
                    if slope > 0:
                        adjusted_fuel_efficiency = fuel_efficiency_per_100km * (1 + slope * 0.1)
                    elif slope < 0:
                        adjusted_fuel_efficiency = fuel_efficiency_per_100km * (1 -abs(slope) * 0.1)
                    else:
                        adjusted_fuel_efficiency = fuel_efficiency_per_100km 

                    fuel_consumption = (travel_distance * adjusted_fuel_efficiency) / 100
                    fuel_consumption = apply_long_distance_optimization(travel_distance, fuel_consumption)
                    return max(fuel_consumption,base_fuel_consumption)
                else:
                    return apply_long_distance_optimization(travel_distance, base_fuel_consumption)
            
        else:
            return np.random.randint(10,100)
    except Exception as e:
        print(f"Error fetching data for {origin_coords} to {destination_coords}: {e}")
        return np.random.randint(10,100)
    

# Function to create a connected graph using real-time data for the selected metric
def create_real_world_graph(locations, coordinates, metric):
    num_nodes = len(locations)
    G = nx.complete_graph(num_nodes)

    for u in range(num_nodes):
        for v in range (u + 1, num_nodes):
            data_value = get_real_time_data(coordinates[u], coordinates[v], metric)
            print(f"Edge ({u}, {v}): {data_value} {metric}")
            G.edges[u,v]['weight'] = data_value
    return G

# Function to generate a route link for the entire route with all waypoints
def generate_full_route_link(tsp_path, coordinates):
    origin_coords = coordinates[tsp_path[0]]
    destination_coords = coordinates[tsp_path[0]]
    waypoints = "|".join([f"{coordinates[node][0]},{coordinates[node][1]}" for node in tsp_path[1:-1]])  # Intermediate waypoints
    
    return f"https://www.google.com/maps/dir/?api=1&origin={origin_coords[0]},{origin_coords[1]}&destination={destination_coords[0]},{destination_coords[1]}&waypoints={waypoints}&travelmode=driving"

# Function to print TSP path details
def print_tsp_details_with_full_link(G, tsp_path, metric, coordinates):
    total_value = 0
    print(f"TSP Path optimized for {metric}:")
    for i in range(len(tsp_path) -1):
        u = tsp_path[i]
        v = tsp_path[i + 1]
        value = G.edges[u, v]['weight']
        total_value += value
        print(f"From {locations[u]} to {locations[v]}: {value:.2f} {'minutes' if metric == 'time' else 'km' if metric == 'distance' else 'liters'}")
    
    if tsp_path[-1] != tsp_path[0]:
        return_trip_value = G.edges[tsp_path[-1], tsp_path[0]]['weight']
        total_value += return_trip_value
        print(f"From {locations[tsp_path[-1]]} to {locations[tsp_path[0]]}: {return_trip_value:.2f} {'minutes' if metric == 'time' else 'km' if metric == 'distance' else 'liters'}")
    
    print(f"Total {metric.capitalize()}: {total_value:.2f} {'minutes' if metric == 'time' else 'km' if metric == 'distance' else 'liters'}")

    google_maps_link = generate_full_route_link(tsp_path, coordinates)
    print(f"View full route on Google Maps: {google_maps_link}")

# Get latitude and longitude
def get_lat_long(location):
    geocode_result = gmaps.geocode(location)
    if geocode_result:
        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        return lat, lng
    return None

# Get Coordinates for all locations
coordinates = [get_lat_long(location) for location in locations]
print(f"Coordinates for locations: {coordinates}")

# Visualize the graph
def visualize_graph(G, coordinates, path=None, metric='time'):
    plt.figure(figsize=(10, 8))
    
    pos = {i: (coord[1], coord[0]) for i, coord in enumerate(coordinates) if coord is not None}

    nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=800)
    nx.draw_networkx_edges(G, pos, edge_color='lightgray', width=1)
    
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)
    
    if path:
        edge_list = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
        if path[-1] != path[0]:  # Add the return trip to the start point
            edge_list.append((path[-1], path[0]))
        nx.draw_networkx_edges(G, pos, edgelist=edge_list, edge_color='red', width=2)
    
    # Kısaltılmış etiketler
    short_labels = {i: f"Loc {i+1}" for i in range(len(locations))}
    nx.draw_networkx_labels(G, pos, short_labels, font_size=12)
    
    plt.title(f"TSP Path Optimized for {metric.capitalize()}")
    plt.show()

# Function to find the TSP path
def find_tsp_path(G):
    tsp_path = traveling_salesman_problem(G, cycle=True, weight='weight')
    print(f"TSP Path found: {tsp_path}")
    return tsp_path

def main():
    metric = input("Choose optimization: 'time', 'distance', or 'fuel': ").lower()
    G = create_real_world_graph(locations, coordinates, metric)
    visualize_graph(G, coordinates, metric=metric)

    tsp_path = find_tsp_path(G)
    print(f"Approximate TSP path: {tsp_path}")

    visualize_graph(G,coordinates, path=tsp_path, metric=metric)
    print_tsp_details_with_full_link(G, tsp_path, metric, coordinates)

if __name__ == "__main__":
    main()